**Replacment of `tf.constant`**

`tf.constant` Creates a [constan tensor](https://www.tensorflow.org/api_docs/python/tf/compat/v1/constant) that can be replaced with simple `torch.Tensor`

In [24]:
import torch
import torch.nn as nn
import tensorflow as tf

In [9]:
name_embeds_t = torch.Tensor([[0.4, 0.3],[0.3, 0.2]])
with tf.variable_scope('name_view' + 'embeddings'):
    name_embeds = tf.constant([[0.4, 0.3],[0.3, 0.2]], dtype=tf.float32)

In [10]:
print(name_embeds, name_embeds_t)

Tensor("name_viewembeddings_2/Const:0", shape=(2, 2), dtype=float32) tensor([[0.4000, 0.3000],
        [0.3000, 0.2000]])


**Replacement of `xavier_init` defined by `MultiKE`**

that uses `tf.variable_scope` which is a context manager for defining ops that creates variables (layers).

The following means that a layer is supposed to be initialized with the [specified](http://tensorflow.biotecan.com/python/Python_1.8/tensorflow.google.cn/api_docs/python/tf/contrib/layers/xavier_initializer.html) initializer and if the corresponding param is set, normalized.

In [20]:
def xavier_init(shape, name, is_l2_norm, dtype=None):
    with tf.name_scope('xavier_init'):
        embeddings = tf.get_variable(name, shape=shape, dtype=dtype,
                                     initializer=tf.contrib.layers.xavier_initializer(uniform=False))
    return tf.nn.l2_normalize(embeddings, 1) if is_l2_norm else embeddings

# https://docs.w3cub.com/tensorflow~python/tf/nn/l2_normalize

In [98]:
# that is supposed to run once and then raise ValueError
with tf.variable_scope('relation_view' + 'embeddings'):
    rv_ent_embeds = xavier_init([179, 75], 'rv_ent_embeds', True)

ValueError: Variable relation_viewembeddings/rv_ent_embeds already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/home/kiril/miniconda3/envs/labenv37/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py", line 1748, in __init__
    self._traceback = tf_stack.extract_stack()
  File "/home/kiril/miniconda3/envs/labenv37/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py", line 3426, in _create_op_internal
    op_def=op_def)
  File "/home/kiril/miniconda3/envs/labenv37/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py", line 3357, in create_op
    attrs, op_def, compute_device)
  File "/home/kiril/miniconda3/envs/labenv37/lib/python3.7/site-packages/tensorflow_core/python/util/deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "/home/kiril/miniconda3/envs/labenv37/lib/python3.7/site-packages/tensorflow_core/python/framework/op_def_library.p

The xavier above has `uniform = False` which means the initialiser will use noral distribution. Below `xavier_normal_` is applied

In [43]:
class Linear(nn.Module):
    def __init__(self, in_features, out_features, bias=True, is_l2_norm=False):
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = torch.nn.Parameter(torch.Tensor(out_features, in_features)) # linear layer
        if(is_l2_norm): # check if it must be normalized.
            self.weight = nn.functional.normalize(self.weight, dim=1, p=2) # do so
        # assuming that this if-else is obsolete:
        if bias: # see https://discuss.pytorch.org/t/tensorflow-converging-faster-than-pytorch/85640
            self.bias = torch.nn.Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
    def reset_parameters(self):
        torch.nn.init.xavier_normal_(self.weight) # https://pytorch.org/docs/stable/nn.init.html
        if self.bias is not None:
            torch.nn.init.uniform_(self.bias, -1, 1)
    def forward(self, input):
        return torch.nn.functional.linear(input, self.weight, self.bias)
        # or one can do a look-up for the embedding here.

Based on the following links
 - [big code snippets](https://discuss.pytorch.org/t/tensorflow-converging-faster-than-pytorch/85640)
 - [small code snippet](https://discuss.pytorch.org/t/how-to-initialize-the-conv-layers-with-xavier-weights-initialization/8419)
 - [l2 normalizer replacement](https://discuss.pytorch.org/t/how-to-implement-batch-l2-normalization-with-pytorch/39707)
 - [tf xavier doc](http://tensorflow.biotecan.com/python/Python_1.8/tensorflow.google.cn/api_docs/python/tf/contrib/layers/xavier_initializer.html)
 - [torch xavier doc](https://pytorch.org/docs/stable/nn.init.html) 

In [44]:
f = Linear(179, 95, bias = False)

**Replacement for `tf.get_variable`**

In [ ]:
with tf.variable_scope('shared' + 'combination'):
            self.nv_mapping = tf.get_variable('nv_mapping', shape=[self.args.dim, self.args.dim],
                                              initializer=tf.initializers.orthogonal())

 - [short example with comment for autograd](https://discuss.pytorch.org/t/tensorflow-get-variable-into-pytorch/83456)
 - as above, but with simplier code and initialzier is orthogonal.

In [102]:
class CharEmbeddings(nn.Module):
    def __init__(self, in_features, out_features):
        # TODO before paste
        super(CharEmbeddings, self).__init__()
        
        emb = torch.empty(in_features, out_features)
        nn.init.orthogonal_(emb)
        print(emb)
        self.nv_mapping = nn.Parameter(emb)

    def forward(self, input):
        return self.nv_mapping[input] # as mentioned in the analogous snippet above.


In [103]:
CharEmbeddings(4,4).forward(0)

tensor([[-0.5650, -0.2847, -0.0924, -0.7689],
        [ 0.1821, -0.1216,  0.9543, -0.2034],
        [-0.7954,  0.0284,  0.2708,  0.5414],
        [-0.1222,  0.9504,  0.0863, -0.2725]])


tensor([-0.5650, -0.2847, -0.0924, -0.7689], grad_fn=<SelectBackward>)

**`tf.placeholder` replacement**

`tf.placeholder` is not needed in torch as one can pass tensors to modules.

**`tf.embedding_lookup` replacement**

In [105]:
with tf.name_scope('relation_triple_placeholder'):
    rel_pos_hs = tf.placeholder(tf.int32, shape=[None])
        
with tf.name_scope('relation_triple_lookup'):
    rel_phs = tf.nn.embedding_lookup(rv_ent_embeds, rel_pos_hs)

Either or:

In [117]:
class CharEmbeddings(nn.Module):
    def __init__(self, in_features, out_features):
        # TODO before paste
        super(CharEmbeddings, self).__init__()
        
        emb = torch.empty(in_features, out_features)
        nn.init.orthogonal_(emb)
        print(emb)
        self.nv_mapping = nn.Parameter(emb)

    def forward(self, input):
        return self.nv_mapping[input] # as mentioned in the analogous snippet above.

emb = nn.Embedding(10, 100)
x = torch.tensor([1,2])
out = emb(x)

In [125]:
x = torch.tensor([0])
emb(x)

tensor([[-0.2863,  0.7333, -0.6096, -1.9199, -2.1164,  0.1582,  0.2363, -3.3401,
         -0.7276,  1.0803, -0.8376,  0.8346, -0.5104, -1.1642, -1.5005,  0.4684,
          0.1012,  1.5001, -0.8946,  0.1354, -1.5158,  0.9176, -2.0890,  3.2442,
         -0.0109, -0.2588, -1.6685, -0.9931,  0.2125, -0.3699, -0.3966, -1.3968,
         -0.0060, -1.2941, -0.7010,  2.1114, -0.5782, -0.6943,  0.8156,  1.4890,
          0.0225, -2.1523,  0.1722,  0.4665,  0.1411,  0.6178, -0.9796,  1.0282,
         -0.6518, -0.7431,  0.4624,  0.9266,  0.2300, -0.3015,  0.2870,  1.1188,
         -0.2594,  0.6772,  0.2805,  0.7594,  0.9710,  0.8420, -0.2592, -1.0879,
         -1.1377, -0.0314, -0.2035, -0.6956, -0.9920,  0.8346,  1.1025,  0.9616,
         -0.2370, -0.0431,  0.1949,  0.0224, -1.9532,  0.2481, -0.0801, -1.1726,
         -1.2579,  0.2772,  0.2767, -1.2489,  0.1103, -0.7892,  2.6883,  0.5771,
          2.3581,  0.1723,  0.6556, -0.5036, -0.3731, -0.3223, -0.7230,  0.1607,
          1.4014, -0.3316,  

### Losses

In [128]:
#Example
def my_loss(output, target):
    loss = torch.mean((output - target)**2)
    return loss

model = nn.Linear(2, 2)
x = torch.randn(1, 2)
target = torch.randn(1, 2)
output = model(x)
loss = my_loss(output, target)
loss.backward()
print(model.weight.grad)

tensor([[-0.3627,  0.4402],
        [ 0.0935, -0.1135]])


In [127]:
def relation_logistic_loss(phs, prs, pts, nhs, nrs, nts):
    pos_distance = phs + prs - pts
    neg_distance = nhs + nrs - nts
    pos_score = -tf.reduce_sum(tf.square(pos_distance), axis=1)
    neg_score = -tf.reduce_sum(tf.square(neg_distance), axis=1)
    pos_loss = tf.reduce_sum(tf.log(1 + tf.exp(-pos_score)))
    neg_loss = tf.reduce_sum(tf.log(1 + tf.exp(neg_score)))
    loss = tf.add(pos_loss, neg_loss)
    return loss

In [130]:
def loss(phs, prs, pts, nhs, nrs, nts):
    pos_distance = phs + prs - pts
    neg_distance = nhs + nrs - nts
    pos_score = -torch.sum(torch.square(pos_distance), dim=1)
    neg_score = -torch.sum(torch.square(neg_distance), dim=1)
    pos_loss = torch.sum(torch.log(1 + torch.exp(-pos_score)))
    neg_loss = torch.sum(torch.log(1 + torch.exp(neg_score)))
    loss = torch.add(pos_loss, neg_loss)
    return loss